In [1]:
import sys
sys.path.append('../')

import gc
import numpy as np
from scipy import ndimage

from skimage.filters import sobel_h
from skimage.filters import sobel_v
from scipy import stats

#from sa_decomp_layer import SADecompLayer



import os
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec


#tf.__version__
from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

plt.rcParams['figure.figsize'] = [10,10]

import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import decode_predictions
from tensorflow.keras.applications import VGG16, ResNet152, ResNet50

from tensorflow.nn import depthwise_conv2d
from tensorflow.math import multiply, reduce_sum, reduce_euclidean_norm, sin, cos, abs
from tensorflow import stack, concat, expand_dims

import tensorflow_probability as tfp
from glob import glob



from tensorflow.keras.utils import to_categorical
from tensorflow.data import Dataset, AUTOTUNE
from utils.imagenet_utils.imagenet_clsloc2 import clsloc
from utils.imagenet_utils.load_images import load_images
from utils.imagenet_utils.preprocess import preprocess

from utils.utils import *

import keras
from keras import layers

from keras import backend as K

from convert_to_3_params import convert_model, NConv2D

In [2]:
gc.collect()

model = ResNet50(include_top=True,
				weights='imagenet',
				classes=1000,
				classifier_activation='softmax')

model2 = convert_model(model, n=2)
layers = [l for l in model2.layers]

opt = keras.optimizers.SGD(learning_rate=0.00001, momentum=0.9)

model2.compile(loss=keras.losses.CategoricalCrossentropy(from_logits=False),    
            optimizer=opt,        
			metrics=[
				keras.metrics.CategoricalAccuracy(name="accuracy"),
				keras.metrics.TopKCategoricalAccuracy(5, name="top-5-accuracy"),])

KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'")
KerasTensor(type_spec=TensorSpec(shape=(None, 230, 230, 3), dtype=tf.float32, name=None), name='conv1_pad/Pad:0', description="created by layer 'conv1_pad'")
KerasTensor(type_spec=TensorSpec(shape=(None, 112, 112, 64), dtype=tf.float32, name=None), name='conv1_conv/BiasAdd:0', description="created by layer 'conv1_conv'")
KerasTensor(type_spec=TensorSpec(shape=(None, 112, 112, 64), dtype=tf.float32, name=None), name='conv1_bn/FusedBatchNormV3:0', description="created by layer 'conv1_bn'")
KerasTensor(type_spec=TensorSpec(shape=(None, 112, 112, 64), dtype=tf.float32, name=None), name='conv1_relu/Relu:0', description="created by layer 'conv1_relu'")
KerasTensor(type_spec=TensorSpec(shape=(None, 114, 114, 64), dtype=tf.float32, name=None), name='pool1_pad/Pad:0', description="created by layer 'pool1_pad'")
KerasTensor(type_spec=TensorSpec(

In [28]:
train_list = glob("C:/imagenette2/train/*/*.JPEG")
train_labels = glob("C:/imagenette2/train/*/*.JPEG")
train_labels = to_categorical([clsloc[os.path.normpath(str(path)).split(os.path.sep)[-2]] for path in train_labels], num_classes = 1000)


trainDS = Dataset.from_tensor_slices((train_list, train_labels))
trainDS = (trainDS
	.map(load_images)
	.map(preprocess)
	.cache()
	.batch(8)
	.prefetch(AUTOTUNE)
)


val_list = glob("C:/imagenette2/val/*/*.JPEG")
val_labels = glob("C:/imagenette2/val/*/*.JPEG")
val_labels = to_categorical([clsloc[os.path.normpath(str(path)).split(os.path.sep)[-2]] for path in val_list], num_classes = 1000)


valDS = Dataset.from_tensor_slices((val_list, val_labels))
valDS = (valDS
	.map(load_images)
	.map(preprocess)
	.cache()
	.batch(32)
	.prefetch(AUTOTUNE)
)

In [29]:
history = model.fit_generator(trainDS,
			epochs=10, 
			validation_data=valDS,
			) 


c:\Users\AN46710\Anaconda3\envs\masters\lib\site-packages\ipykernel_launcher.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  This is separate from the ipykernel package so we can avoid doing imports until


Epoch 1/10
1184/1184 [==============================] - 272s 227ms/step - loss: 0.7705 - accuracy: 0.9089 - top-5-accuracy: 0.9386 - val_loss: 4.6006 - val_accuracy: 0.3330 - val_top-5-accuracy: 0.5817
Epoch 2/10
1184/1184 [==============================] - 212s 179ms/step - loss: 0.3256 - accuracy: 0.9428 - top-5-accuracy: 0.9788 - val_loss: 2.1045 - val_accuracy: 0.6176 - val_top-5-accuracy: 0.8736
Epoch 3/10
1184/1184 [==============================] - 206s 174ms/step - loss: 0.1723 - accuracy: 0.9586 - top-5-accuracy: 0.9929 - val_loss: 1.4475 - val_accuracy: 0.6971 - val_top-5-accuracy: 0.9358
Epoch 4/10
1184/1184 [==============================] - 204s 172ms/step - loss: 0.1283 - accuracy: 0.9658 - top-5-accuracy: 0.9966 - val_loss: 1.2155 - val_accuracy: 0.7266 - val_top-5-accuracy: 0.9552
Epoch 5/10
1184/1184 [==============================] - 205s 173ms/step - loss: 0.1026 - accuracy: 0.9716 - top-5-accuracy: 0.9976 - val_loss: 1.1352 - val_accuracy: 0.7437 - val_top-5-accurac

In [30]:
model.layers[2].get_weights()[0][:,:,0,0]

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[ 0.18899006,  0.1168705 ,  0.04475094],
       [ 0.11251222,  0.04039265, -0.03172691],
       [ 0.03603437, -0.0360852 , -0.10820476]], dtype=float32)>

In [31]:
model.layers[2].get_weights()[0][:,:,0,0]

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[ 0.18899006,  0.1168705 ,  0.04475094],
       [ 0.11251222,  0.04039265, -0.03172691],
       [ 0.03603437, -0.0360852 , -0.10820476]], dtype=float32)>